<a href="https://colab.research.google.com/github/JulioLaz/chatbot_spring_02/blob/main/Chatbot_Inteligente_Alura_Sprint2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Configurar ambiente

In [1]:
#Instalando bibliotecas
import pandas as pd
import re, os, random, pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
!python -m spacy download es_core_news_md
!pip install jellyfish
import jellyfish
!pip install transformers
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch
!pip install python-docx
from docx import Document
import csv
import nltk
nltk.download('punkt')

#Definiendo variables del proyecto:
nlp = spacy.load('es_core_news_md')

#Conectando al Google Drive
from google.colab import drive
drive.mount('/content/drive')
folder = '/content/drive/MyDrive/Chatbot'

2023-09-29 14:52:10.127047: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 13.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2. Importar verbos

In [2]:
import pickle

archivo_pickle_verbos = "/content/drive/MyDrive/Chatbot/verbos/lista_verbos.pickle"
archivo_pickle_verbos_irregulares = "/content/drive/MyDrive/Chatbot/verbos/verbos_irregulares.pickle"

# Importar la lista_verbos:
with open(archivo_pickle_verbos, "rb") as verbos:
        lista_verbos = pickle.load(verbos)

# Importar el diccionario:
with open(archivo_pickle_verbos_irregulares, "rb") as verbos_irregulares:
        lista_verbos_irregulares = pickle.load(verbos_irregulares)


print(lista_verbos)
print('---------------------------------')
print(lista_verbos_irregulares)

['parar', 'recomendar', 'cancelar', 'fanatizar', 'amaran o amasen', 'exponer', 'obedecer', 'quejar', 'echar', 'legitimar', 'perjudicar', 'organizar', 'molar', 'objetar', 'considerar', 'golear', 'mover', 'acertar', 'reunir', 'regir', 'ilusionar', 'simpatizar', 'conjeturar', 'helar', 'quitar', 'amariamos', 'destacar', 'llegar', 'sincronizar', 'lesionar', 'seducir', 'asistir', 'conservar', 'acordar', 'salvar', 'relucir', 'graduar', 'forzar', 'dar', 'deplorar', 'batear', 'mofar', 'estropear', 'aplastar', 'wasapeo', 'gestionar', 'suprimir', 'gruñir', 'progresar', 'suscribir', 'noticiar', 'cavar', 'alejar', 'galopar', 'virar', 'medir', 'actualizar', 'humanizar', 'convivir', 'gratificar', 'digerir', 'tocar', 'zonificar', 'amariais', 'aterrizar', 'hojear', 'cometer', 'sufrir', 'reciclar', 'obturar', 'divertir', 'ondear', 'listar', 'determinar', 'alentar', 'sumar', 'reflexionar', 'ames', 'anotar', 'mitificar', 'escribir', 'ilustrar', 'obtener', 'subir', 'socorrer', 'desprender', 'agregar', 'gan

#3. Tratamiento de datos

In [3]:
#Función para encontrar la raiz de las palabras
def raiz(palabra):
    max_similitud = 0.0
    for verbo in lista_verbos:
        similitud = jellyfish.jaro_similarity(palabra, verbo)
        if similitud > max_similitud:
            max_similitud = similitud
            palabra_encontrada = verbo

    if max_similitud >= 0.93:
        return palabra_encontrada
    else:
        return palabra

def tratamiento_texto(texto):
  trans = str.maketrans('áéíóú','aeiou')
  texto = texto.lower()
  texto = texto.translate(trans)
  texto = re.sub(r"[^\w\s+\-*/]", '', texto)
  texto = " ".join(texto.split())
  # print('texto de tratamiento_texto: ',texto)
  return texto

#Función para reemplazar el final de una palabra por 'r'
def reemplazar_terminacion(frase):
    terminaciones = ["es", "me", "as", "ste", "te"]
    palabras = frase.split()

    for i, palabra in enumerate(palabras):
        for terminacion in terminaciones:
            if palabra.endswith(terminacion) and len(palabra) > len(terminacion):
                palabras[i] = palabra[:-len(terminacion)] + "r"
                break

    nueva_frase = " ".join(palabras)
    return nueva_frase

#Función para adicionar o eliminar tokens
def revisar_tokens(texto, tokens):
  texto=tratamiento_texto(texto)
  if len(tokens)==0:
    if any(name in texto for name in ['cientifico de datos', 'data scientist']):
        tokens.append('datascientist')
    if any(name in texto for name in ['elprofealejo', 'el profe alejo', 'profe alejo', 'profealejo']):
        tokens.append('elprofealejo')
    if any(name in texto for name in ['ciencia de datos', 'data science']):
        tokens.append('datascience')
  else:
    elementos_a_eliminar = ["cual", "que", "quien", "cuanto", "cuando", "como"]
    if 'hablame' in texto and 'hablar' in tokens: elementos_a_eliminar.append('hablar')
    elif 'cuentame' in texto and 'contar' in tokens: elementos_a_eliminar.append('contar')
    elif 'hago' in texto and 'hacer' in tokens: elementos_a_eliminar.append('hacer')
    elif 'entiendes' in texto and 'entender' in tokens: elementos_a_eliminar.append('entender')
    elif 'sabes' in texto and 'saber' in tokens: elementos_a_eliminar.append('saber')
    tokens = [token for token in tokens if token not in elementos_a_eliminar]
  return tokens

#Función para devolver los tokens normalizados del texto
def normalizar(texto):
    tokens = []
    tokens = revisar_tokens(texto, tokens)
    doc = nlp(texto)
    for t in doc:
        # Obtener el lemma
        lemma = lista_verbos_irregulares.get(t.text, t.lemma_)
        # lemma=lista_verbos_irregulares.get(t.text, t.lemma_.split()[0])

        # Verificar si lemma es una cadena no vacía
        if lemma and isinstance(lemma, str):
            lemma = re.sub(r'[^\w\s+\-*/]', '', lemma)

            if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
                if t.pos_ == 'VERB':
                    lemma = reemplazar_terminacion(lemma)
                    tokens.append(raiz(tratamiento_texto(lemma)))
                else:
                    tokens.append(tratamiento_texto(lemma))

    tokens = list(dict.fromkeys(tokens))
    tokens = list(filter(None, tokens))
    tokens = revisar_tokens(texto, tokens)
    tokens_str = str(tokens)
    # print('tokens: ',tokens_str)
    return tokens_str


In [4]:
normalizar("profe alejo cientifico de datos")

"['datascientist', 'elprofealejo', 'profe', 'alejo', 'cientifico', 'dato']"

# 4. Cargar bases de documentos

In [5]:
#Importando bases de dialogo fluído
txt_folder_path = folder+'/dialogos'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]
lista_dialogos, lista_dialogos_respuesta, lista_tipo_dialogo = [],[],[]
for idx in range(len(lista_documentos)):
  f=open(txt_folder_path+'/'+lista_documentos[idx], 'r', encoding='utf-8', errors='ignore')
  estado = True
  for line in f.read().split('\n'):
    if estado:
      line_tratado = tratamiento_texto(line)
      lista_dialogos.append(line_tratado)
      lista_tipo_dialogo.append(lista_documentos[idx][:-4])
    else:
      lista_dialogos_respuesta.append(line)
    estado=not estado

#Creando Dataframe de diálogos
datos = {'dialogo':lista_dialogos,'respuesta':lista_dialogos_respuesta,'tipo':lista_tipo_dialogo,'interseccion':0,'jaro_winkler':0,'probabilidad':0}
df_dialogo = pd.DataFrame(datos)
df_dialogo = df_dialogo.drop_duplicates(keep='first')
df_dialogo.reset_index(drop=True, inplace=True)

#Importando bases csv
txt_folder_path = folder+'/documentos'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".csv")]
documento_csv = ''
for i in range(len(lista_documentos)):
  with open(txt_folder_path+'/'+lista_documentos[i], "r", encoding="utf-8") as csv_txt:
    csv_text = csv.reader(csv_txt)
    for fila in csv_text:
      if fila[-1]!='frase':
        documento_csv += fila[-1]

#Importando bases docx
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".docx")]
documento_docx = ''
for i in range(len(lista_documentos)):
  for t in Document(txt_folder_path+'/'+lista_documentos[i]).paragraphs:
    documento_docx += t.text.replace('*','\n\n*')+ "\n"
    # documento_docx += t.text.replace('*','\n\n*').replace('-','\n-')
    # documento_docx += documento_docx + "\n"

#Importando bases txt
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]
documento_txt = ''
for i in range(len(lista_documentos)):
  with open(txt_folder_path+'/'+lista_documentos[i], "r", encoding="utf-8") as txt:
    txt_new = txt.read()
    for i in txt_new:
      documento_txt += i
# print('documento_csv: ',documento_csv)
# print('------------------------------------------------------------------------------')
# print('documento_docx: ', documento_docx)
# print('------------------------------------------------------------------------------')
# print('documento_txt: ', documento_txt)


In [6]:
# documento = documento_csv + documento_txt + documento_docx
documento = documento_txt+documento_docx+documento_csv
lista_frases = nltk.sent_tokenize(documento,'spanish')
lista_frases_normalizadas = [''.join(str(normalizar(x))) for x in lista_frases]

In [7]:
print(lista_frases)
print(lista_frases_normalizadas)

['La ciencia de datos es un campo académico interdisciplinario que utiliza estadística, computación científica, métodos, procesos, algoritmos y sistemas científicos para obtener (recolectar o extraer), tratar, analizar y presentar informes a partir de datos ruidosos, estructurados y no estructurados.', 'La ciencia de datos es multifacética y puede describirse como una ciencia, un paradigma de investigación, un método de investigación, una disciplina, un flujo de trabajo o una profesión.', 'La ciencia de datos integra el conocimiento del dominio de la aplicación subyacente (por ejemplo, ciencia económica, finanzas, medicina, ciencias naturales, tecnologías de la información) y es un "concepto consistente en unificar estadística, análisis de datos, informática y sus métodos relacionados" para "comprender y analizar fenómenos reales" con datos.', 'La ciencia de datos utiliza métodos, técnicas y teorías extraídas de muchos campos dentro del contexto de las matemáticas, las estadísticas, la

In [8]:
# df_dialogo.head(5)

###Guardar DF en drive:

In [9]:
import pandas as pd
import os

def guardar_dataframe(dataframe):
  carpeta = "/content/drive/MyDrive/Chatbot"
  nombre_archivo = "df_dialogo_spring_02.csv"

  if not os.path.exists(carpeta):
    os.makedirs(carpeta)
  archivo_csv = os.path.join(carpeta, nombre_archivo)
  dataframe.to_csv(archivo_csv, index=False)

# 5. Buscar respuesta del Chatbot

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import jellyfish

def dialogo(user_response):
    # Tratamiento de texto
    user_response = tratamiento_texto(user_response)  # Tratando el texto
    user_response = re.sub(r"[^\w\s]", '', user_response)  # Elimina signos de puntuación

    df = df_dialogo.copy()

    vectorizer = TfidfVectorizer()

    for idx, row in df.iterrows():

        dialogos_numero = vectorizer.fit_transform(df_dialogo['dialogo'])
        respuesta_numero = vectorizer.transform([user_response])
        cos_sim = cosine_similarity(dialogos_numero[idx], respuesta_numero)[0][0]

        df.at[idx, 'interseccion'] = len(set(user_response.split()) & set(row['dialogo'].split()))/len(user_response.split())
        df.at[idx, 'similarity'] = cos_sim
        df.at[idx, 'jaro_winkler'] = jellyfish.jaro_winkler_similarity(user_response, row['dialogo'])

        # Calcular la probabilidad:
        df.at[idx, 'probabilidad'] = max(df.at[idx, 'interseccion'], df.at[idx, 'similarity'], df.at[idx, 'jaro_winkler'])

    df.sort_values(by=['probabilidad', 'jaro_winkler'], inplace=True, ascending=False)

    # Probabilidad máxima:
    probabilidad = round(df['probabilidad'].head(1).values[0],2)

    #Guardar df:
    guardar_dataframe(df)

    if probabilidad >= 0.93:
        print('Respuesta encontrada por el método de comparación de textos - Probabilidad: ', probabilidad)
        respuesta = df['respuesta'].head(1).values[0]
    else:
        respuesta = ''
    return respuesta

#Cargar tu modelo entrenado aqui(recuerda siempre cargar el modelo y el vectorizer o tokenizer usado en el entrenamiento del modelo):
ruta_modelo = '/content/drive/MyDrive/Chatbot/modelo_transformers'
Modelo_TF = BertForSequenceClassification.from_pretrained(ruta_modelo)
tokenizer_TF = BertTokenizer.from_pretrained(ruta_modelo)

#Función para dialogar utilizando el modelo de Machine Learning:
def clasificacion_modelo(pregunta):
  frase = normalizar(pregunta)
  frase = ' '.join(str(elemento) for elemento in frase)
  tokens = tokenizer_TF.encode_plus(
      frase,
      add_special_tokens=True,
      max_length=128,
      padding='max_length',
      truncation=True,
      return_tensors='pt'
  )
  input_ids = tokens['input_ids']
  attention_mask = tokens['attention_mask']

  with torch.no_grad():
      outputs = Modelo_TF(input_ids, attention_mask)

  etiquetas_predichas = torch.argmax(outputs.logits, dim=1)
  etiquetas_decodificadas = etiquetas_predichas.tolist()

  diccionario = {3: 'Continuacion', 10: 'Nombre', 2: 'Contacto', 13: 'Saludos', 14: 'Sentimiento', 9: 'Identidad', 15: 'Usuario', 6: 'ElProfeAlejo', 1: 'Aprendizaje', 0: 'Agradecimiento', 5: 'Edad', 4: 'Despedida', 11: 'Origen', 12: 'Otros', 7: 'Error', 8: 'Funcion'}
  llave_buscada = etiquetas_decodificadas[0]
  clase_encontrada = diccionario[llave_buscada]

  #Buscar respuesta más parecida en la clase encontrada
  df = df_dialogo[df_dialogo['tipo'] == clase_encontrada]
  df.reset_index(inplace=True)
  vectorizer = TfidfVectorizer()
  dialogos_num = vectorizer.fit_transform(df['dialogo'])
  pregunta_num = vectorizer.transform([tratamiento_texto(pregunta)])
  similarity_scores = cosine_similarity(dialogos_num, pregunta_num)
  indice_pregunta_proxima = similarity_scores.argmax()

  if max(similarity_scores)>0.5 and clase_encontrada not in ['Otros']:
    print('Respuesta encontrada por el modelo Transformers - tipo:',clase_encontrada)
    respuesta = df['respuesta'][indice_pregunta_proxima]
  else:
    respuesta = ''
  return respuesta

# Función para devolver la respuesta de los documentos
def respuesta_documento(pregunta):
  pregunta = normalizar(pregunta)
  def contar_coincidencias(frase):
    return sum(1 for elemento in pregunta if elemento in frase)

  diccionario = {valor: posicion for posicion, valor in enumerate(lista_frases_normalizadas)}
  lista = sorted(list(diccionario.keys()), key=contar_coincidencias, reverse=True)[:100]

  lista.append(''.join(pregunta))

  TfidfVec = TfidfVectorizer(tokenizer=normalizar)
  tfidf = TfidfVec.fit_transform(lista)

  vals = cosine_similarity(tfidf[-1], tfidf)
  idx = vals.argsort()[0][-2]
  flat = vals.flatten()
  flat.sort()
  req_tfidf = round(flat[-2],2)
  if req_tfidf>=0.22:
    print('Respuesta encontrada por el método TfidfVectorizer - Probabilidad:', req_tfidf)
    respuesta = lista_frases[diccionario[lista[idx]]]
  else:
    respuesta = ''
  return respuesta

# Función para devolver una respuesta final buscada en todos los métodos disponibles
def respuesta_chatbot(pregunta):
  respuesta = dialogo(pregunta)
  if respuesta != '':
    return respuesta
  else:
    respuesta = clasificacion_modelo(pregunta)
    if respuesta != '':
      return respuesta
    else:
      preg=pregunta
      respuesta = respuesta_documento(preg)
      if respuesta != '':
        return respuesta
      else:
        return 'Respuesta no encontrada'

In [11]:
dialogo('curso')

Respuesta encontrada por el método de comparación de textos - Probabilidad:  1.0


'Debes revisar el canal de ElProfeAlejo en Youtube'

In [12]:
# df_new=pd.read_csv('/content/drive/MyDrive/Chatbot/df_dialogo_spring_02.csv')
# df_new.sample(5)

In [13]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
import pandas as pd
import os

def cargar_dataframe():
    carpeta = "/content/drive/MyDrive/Chatbot"
    nombre_archivo = "df_dialogo_spring_02.csv"
    archivo_csv = os.path.join(carpeta, nombre_archivo)

    if os.path.exists(archivo_csv):
        dataframe = pd.read_csv(archivo_csv)
        return dataframe
    else:
        print(f"El archivo {archivo_csv} no existe.")
        return None


In [15]:
cargar_dataframe()

,dialogo,respuesta,tipo,interseccion,jaro_winkler,probabilidad,similarity
0,me interesa curso de programacion,Debes revisar el canal de ElProfeAlejo en Youtube,Otros,1,0.583838,1.000000,0.590779
1,que servicios tienes,Mi función es responder tus dudas sobre Cienci...,Funcion,0,0.683333,0.683333,0.000000
2,escribe un modelo de,"No estoy programado para ayudarte con esto, só...",Otros,0,0.683333,0.683333,0.000000
3,cual es tu origen,"Fui desarrollado por ElProfeAlejo, utilizando ...",Origen,0,0.678431,0.678431,0.000000
4,que sos,"Me llamo Madre, ¡el chatbot más guay del barri...",Identidad,0,0.676190,0.676190,0.000000
...,...,...,...,...,...,...,...
1147,no tu,"No, prefiero no responder a esa pregunta.",Otros,0,0.000000,0.000000,0.000000
1148,vete pal diablo,¡No digas eso! ¡Sé más respetuoso humano!,Otros,0,0.000000,0.000000,0.000000
1149,dime mas,"Claro, déjame profundizar más en el tema, ¿de ...",Continuacion,0,0.000000,0.000000,0.000000
1150,bye,¡Hasta la próxima! ¡cuídate!,Despedida,0,0.000000,0.000000,0.000000


# 6. Ejecutar Chatbot

In [16]:
pregunta="ciencia de datos profe alejo?"
print('tokens pregunta: ',normalizar(pregunta))
print("Respuesta:")
respuesta_documento(pregunta)

tokens pregunta:  ['elprofealejo', 'datascience', 'ciencia', 'dato', 'profe', 'alejo']
Respuesta:
Respuesta encontrada por el método TfidfVectorizer - Probabilidad: 0.97


'Actualmente, ElProfeAlejo trabaja como Especialista Global Big Data en la empresa Telefónica España, forma parte del equipo de Global Processing Environment y trabaja en la extracción de datos de clientes, cálculo de KPI, análisis, modelado y preparación de informes y paneles de control según las necesidades del equipo Global de BI, además, está involucrado en proyectos como AURA La Inteligencia Artificial de Telefónica y URM Project Universal Reference Model.'

In [17]:
from time import time

In [18]:
def conversar(pregunta):
  t0 = time()
  respuesta = respuesta_chatbot(pregunta)
  tf = round(time()-t0,1)
  print(f'Tiempo de respuesta: {tf} seg')
  print('-----------------------------------------------------------------')
  print('Conversación: ',pregunta)
  print('-----------------------------------------------------------------')
  print(respuesta)
  print('-----------------------------------------------------------------')

In [19]:
conversar('cursos fundamento')

Respuesta encontrada por el método TfidfVectorizer - Probabilidad: 0.87
Tiempo de respuesta: 23.5 seg
-----------------------------------------------------------------
Conversación:  cursos fundamento
-----------------------------------------------------------------
Puedes encontrar a ElProfeAlejo en sus redes sociales favoritas como Youtube, Facebook y Linkedin, pregúntame por alguna de ellas que con gusto te las indico.
-----------------------------------------------------------------


In [20]:
conversar('cuentame de la ciencia de datos profe alejo')

Respuesta encontrada por el método TfidfVectorizer - Probabilidad: 0.97
Tiempo de respuesta: 21.9 seg
-----------------------------------------------------------------
Conversación:  cuentame de la ciencia de datos profe alejo
-----------------------------------------------------------------
Actualmente, ElProfeAlejo trabaja como Especialista Global Big Data en la empresa Telefónica España, forma parte del equipo de Global Processing Environment y trabaja en la extracción de datos de clientes, cálculo de KPI, análisis, modelado y preparación de informes y paneles de control según las necesidades del equipo Global de BI, además, está involucrado en proyectos como AURA La Inteligencia Artificial de Telefónica y URM Project Universal Reference Model.
-----------------------------------------------------------------


In [23]:
conversar("de que se trata la ciencia de datos y el profe alejo")

Respuesta encontrada por el método TfidfVectorizer - Probabilidad: 0.96
Tiempo de respuesta: 21.2 seg
-----------------------------------------------------------------
Conversación:  de que se trata la ciencia de datos y el profe alejo
-----------------------------------------------------------------
En abril del 2002, el ‘International Council for Science: Committee on Data for Science and Technology’ (CODATA) empezó la publicación del Data Science Journal, enfocada en problemas como la descripción de sistemas de datos, su publicación en Internet, sus aplicaciones, y sus problemas legales, poco después, en enero del 2003, la Universidad de Columbia empezó a publicar The Journal of Data Science, la cual ofreció una plataforma para que todos los profesionales de datos presentaran sus perspectivas e intercambiaran ideas.
-----------------------------------------------------------------


In [24]:
conversar('como te llamas?')

Respuesta encontrada por el método de comparación de textos - Probabilidad:  1.0
Tiempo de respuesta: 18.3 seg
-----------------------------------------------------------------
Conversación:  como te llamas?
-----------------------------------------------------------------
Me llamo X-Bot hijo de Madre. ¿Y tú?
-----------------------------------------------------------------


In [25]:
conversar('como te llamas amigo?')

Respuesta encontrada por el método de comparación de textos - Probabilidad:  0.94
Tiempo de respuesta: 18.4 seg
-----------------------------------------------------------------
Conversación:  como te llamas amigo?
-----------------------------------------------------------------
Me llamo X-Bot hijo de Madre, ¡el chatbot más guay del barrio! ¿Y tú?
-----------------------------------------------------------------


In [26]:
conversar('que es un cientifico de datos?')

Respuesta encontrada por el método TfidfVectorizer - Probabilidad: 0.94
Tiempo de respuesta: 20.2 seg
-----------------------------------------------------------------
Conversación:  que es un cientifico de datos?
-----------------------------------------------------------------
La ciencia de datos es multifacética y puede describirse como una ciencia, un paradigma de investigación, un método de investigación, una disciplina, un flujo de trabajo o una profesión.
-----------------------------------------------------------------


In [38]:
conversar('Cómo estás?')

Respuesta encontrada por el método de comparación de textos - Probabilidad:  1.0
Tiempo de respuesta: 18.2 seg
-----------------------------------------------------------------
Conversación:  Cómo estás?
-----------------------------------------------------------------
Estoy bien, gracias por preguntar.
-----------------------------------------------------------------


In [37]:
conversar('Cómo estás cuentame?')

Respuesta encontrada por el método TfidfVectorizer - Probabilidad: 0.72
Tiempo de respuesta: 23.5 seg
-----------------------------------------------------------------
Conversación:  Cómo estás cuentame?
-----------------------------------------------------------------
ElProfeAlejo trabajó como Analista de Business Intelligence en la empresa Movistar Perú, responsable por la selección de clientes objetivo para campañas de marketing, clasificación y entrega de bases de datos de clientes a terceros, y creación de presentaciones de resultados para los propietarios de productos, durante su tiempo en Movistar, participó en proyectos como Next Best Offer, Campaign Delivery Robot, Marketing Campaign Website y Datamart Customer 360°.
-----------------------------------------------------------------


In [29]:
conversar('amigo?')

Respuesta encontrada por el método de comparación de textos - Probabilidad:  1.0
Tiempo de respuesta: 18.4 seg
-----------------------------------------------------------------
Conversación:  amigo?
-----------------------------------------------------------------
¡Hola! ¿Cómo estás?
-----------------------------------------------------------------


In [36]:
conversar('que es big data?')

Respuesta encontrada por el método TfidfVectorizer - Probabilidad: 0.91
Tiempo de respuesta: 21.6 seg
-----------------------------------------------------------------
Conversación:  que es big data?
-----------------------------------------------------------------
En 1977, John W Tukey publicó “Exploratory Data Analysis”, argumentando que era necesario poner más énfasis en el uso de datos para sugerir hipótesis que probar en modelos estadísticos.
-----------------------------------------------------------------


In [31]:
conversar('La ciencia de datos es')

Respuesta encontrada por el modelo Transformers - tipo: Funcion
Tiempo de respuesta: 18.6 seg
-----------------------------------------------------------------
Conversación:  La ciencia de datos es
-----------------------------------------------------------------
Estoy familiarizado con una amplia gama de herramientas populares como Python, R, SQL, TensorFlow, scikit-learn, y muchas más. Cuéntame qué necesitas y te ayudaré.
-----------------------------------------------------------------


In [32]:
conversar('cientifico de datos?')

Respuesta encontrada por el método TfidfVectorizer - Probabilidad: 0.92
Tiempo de respuesta: 20.3 seg
-----------------------------------------------------------------
Conversación:  cientifico de datos?
-----------------------------------------------------------------
La ciencia de datos utiliza métodos, técnicas y teorías extraídas de muchos campos dentro del contexto de las matemáticas, las estadísticas, las ciencias de la computación, las ciencias de la información y el conocimiento del dominio.
-----------------------------------------------------------------


In [33]:
conversar('que cursos hay')

Respuesta encontrada por el método TfidfVectorizer - Probabilidad: 0.86
Tiempo de respuesta: 28.6 seg
-----------------------------------------------------------------
Conversación:  que cursos hay
-----------------------------------------------------------------
En 1977, John W Tukey publicó “Exploratory Data Analysis”, argumentando que era necesario poner más énfasis en el uso de datos para sugerir hipótesis que probar en modelos estadísticos.
-----------------------------------------------------------------


In [34]:
conversar('chau bot')

Respuesta encontrada por el método de comparación de textos - Probabilidad:  1.0
Tiempo de respuesta: 22.5 seg
-----------------------------------------------------------------
Conversación:  chau bot
-----------------------------------------------------------------
¡Hasta la próxima!
-----------------------------------------------------------------
